In [3]:
# Import necessary libraries

import requests
import lxml.html as lh
import pandas as pd

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')


#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

#Creating Pandas DataFrame

#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1


# Check the length of each column. Ideally, they should all be the same
[len(C) for (title,C) in col]

Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

df.head()

df.columns = ['Postcode','Borough', 'Neighbourhood']

cols = df.columns.tolist()
cols

cols = cols[-1:] + cols[:-1]
cols
df = df[cols]

df.head()

df = df.replace('\n',' ', regex=True)
df.head()

df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

df.head(10)

df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(20)

df['Neighbourhood'] = df['Neighbourhood'].str.strip()

df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df['Postcode']=df['Postcode'].str.replace(" ","")
df.shape

df.head()

1:"Postal code
"
2:"Borough
"
3:"Neighborhood
"


,Postcode,Borough,Neighbourhood
0,,Canadian postal codes,
1,M1A,Not assigned,
2,M1B,Scarborough,Malvern / Rouge
3,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
4,M1E,Scarborough,Guildwood / Morningside / West Hill


In [15]:
df.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [5]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

link = "http://cocl.us/Geospatial_data"
df1 = pd.read_csv(link)

df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
df1.shape

(103, 3)

In [7]:
df1.columns = ['Postcode','Latitude','Longitude']

cols = df1.columns.tolist()
cols

['Postcode', 'Latitude', 'Longitude']

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 3 columns):
Postcode         181 non-null object
Borough          181 non-null object
Neighbourhood    181 non-null object
dtypes: object(3)
memory usage: 4.3+ KB


In [17]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
Postcode     103 non-null object
Latitude     103 non-null float64
Longitude    103 non-null float64
dtypes: float64(2), object(1)
memory usage: 2.5+ KB


In [24]:

df_new = pd.merge(df, df1, on='Postcode',how='inner')
df_new.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
